Need to process streets into links and nodes, with nodes as intersections

In [23]:
import os
import pandas as pd

from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz,MapViz

streets = pd.read_csv('nyc_streets.csv')

In [24]:
streets.head()

,street,name,type,oneway,node,lat,lon
0,5668966,West 106th Street,secondary,NaN,4205830390,40.798208,-73.960425
1,5668966,West 106th Street,secondary,NaN,3602678205,40.798324,-73.960705
2,5668966,West 106th Street,secondary,NaN,4205830391,40.798485,-73.961090
3,5668968,West 80th Street,residential,yes,42421778,40.785593,-73.982011
4,5668968,West 80th Street,residential,yes,8727756094,40.785083,-73.980801


An intersecion is a node with shared streets

### Useful info
The nodes defining the geometry of the way are enumerated in the correct order, and indicated only by reference using their unique identifier. These nodes must have been already defined separately with their coordinates. 

### Tasks
- find nodes that are intersections
- removed intersections at the same geographical locations
- 

In [25]:
# get nodes that appear more than once - assume these are intersections
node_counts = streets['node'].value_counts()
inters = node_counts[node_counts.values > 1]
len(inters)

5476

In [27]:
# by joining the inters on the streets, we should only have the interstctions 
# of each street remaining
street_links = streets.join(
    inters.rename("intersection"),how="inner",on=["node"]
).rename_axis("order").sort_values(["street","order"])

# sort by street and index to get the links
print(max(street_links['intersection']))
street_links.head(10)

6


,street,name,type,oneway,node,lat,lon,intersection
order,,,,,,,,
0,5668966,West 106th Street,secondary,NaN,4205830390,40.798208,-73.960425,2
2,5668966,West 106th Street,secondary,NaN,4205830391,40.798485,-73.961090,2
3,5668968,West 80th Street,residential,yes,42421778,40.785593,-73.982011,2
5,5668968,West 80th Street,residential,yes,42421776,40.785031,-73.980678,3
8,5668968,West 80th Street,residential,yes,42421775,40.784569,-73.979583,2
9,5668968,West 80th Street,residential,yes,1061531525,40.784494,-73.979405,2
12,5668968,West 80th Street,residential,yes,42421772,40.783828,-73.977825,2
13,5668968,West 80th Street,residential,yes,42421769,40.782638,-73.975007,2
14,5668973,Hillside Avenue,tertiary,yes,8904913030,40.859078,-73.931443,3


In [31]:
# let's vizualise this misterious street
df_to_geojson(street_links.loc[:,['node','lat','lon','intersection']]
              , filename='intersections.geojson',
              properties=['intersection'],
              lat='lat', lon='lon', precision=3)

# Generate data breaks and color stops from colorBrewer
color_breaks = [2,3,4,5,6]
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

# Create the viz from the dataframe
token = os.getenv('MAPBOX_TOKEN')
viz = CircleViz('intersections.geojson',
                access_token=token,
                height='400px',
                color_stops=color_stops,
#                 stroke_width=3,
                center=(-73.969,40.766),
                zoom = 15,
                below_layer = 'waterway-label'
              )
viz.show()